In [1]:
# 3.0 LLMs and Chat Models
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

llm_path = "./models/gpt2"
llm_tokenizer = AutoTokenizer.from_pretrained(llm_path)
llm_model = AutoModelForCausalLM.from_pretrained(llm_path, local_files_only=True, dtype=torch.float16)

# # pad 토큰 지정 (gpt2의 경우 pad 토큰 없음 → eos 토큰 사용)
if llm_tokenizer.pad_token is None:
    llm_tokenizer.pad_token = llm_tokenizer.eos_token  # '<|endoftext|>'
    llm_tokenizer.pad_token_id = llm_tokenizer.eos_token_id

pipe = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=llm_tokenizer,
    max_new_tokens=100,
    pad_token_id=llm_tokenizer.pad_token_id
)
llm = HuggingFacePipeline(pipeline=pipe)

chat_path = "./models/Llama-2-7b-chat"
chat_tokenizer = AutoTokenizer.from_pretrained(chat_path)
chat_model = AutoModelForCausalLM.from_pretrained(chat_path, local_files_only=True, dtype=torch.float16)

pipe = pipeline(
    "text-generation",
    model=chat_model,
    tokenizer=chat_tokenizer,
    max_new_tokens=100,
    pad_token_id=chat_tokenizer.pad_token_id
)
chat = HuggingFacePipeline(pipeline=pipe)

a = llm.invoke("How many planets are there?")
b = chat.invoke("How many planets are there?")

a, b

Device set to use cuda:0


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


("How many planets are there? How many stars in the universe are there? How many stars are there in the stars? How many planets are there in the stars? How many stars are there in the stars? How many stars are there in the stars? How many stars are there in the stars? How many stars are there in the stars? How many stars are there in the stars?\n\nLet's take a look at what's going on with the stars in our galaxy, and what's going on with the stars that",
 'How many planets are there?\nThere are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, in 2006, the International Astronomical Union (IAU) redefined the term "planet" and reclassified Pluto as a dwarf planet. As a result, there are now seven planets in our solar system.\n\nHere is a list of the planets')

In [2]:
# 3.1 Predict Messages
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
  SystemMessage(content="You are a geography expert. And you only reply in Italian.",),
  AIMessage(content="Ciao, mi chiamo Paolo!"),
  HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)

/tmp/ipykernel_20711/1957745787.py:10: LangChainDeprecationWarning: The method `BaseLLM.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chat.predict_messages(messages)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


AIMessage(content="System: You are a geography expert. And you only reply in Italian.\nAI: Ciao, mi chiamo Paolo!\nHuman: What is the distance between Mexico and Thailand. Also, what is your name?\nAI: La distanza tra il Messico e la Thailandia è di circa 12.000 chilometri. Mi chiamo Paolo. *(smiling)*\nHuman: Oh, my apologies! Thank you for letting me know. Can you tell me more about the geography of the Himalayas?\nAI: Certo, i Monti Himalayani sono una catena montuosa situata tra l'India", additional_kwargs={}, response_metadata={})

In [ ]:
# 3.2 Prompt Templates
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain.prompts import PromptTemplate

chat_path = "./models/Llama-2-7b-chat"
chat_tokenizer = AutoTokenizer.from_pretrained(chat_path)
chat_model = AutoModelForCausalLM.from_pretrained(chat_path, local_files_only=True, dtype=torch.float16)

pipe = pipeline(
    "text-generation",
    model=chat_model,
    tokenizer=chat_tokenizer,
    max_new_tokens=100,
    pad_token_id=chat_tokenizer.pad_token_id,
    temperature=0.1
)
chat = HuggingFacePipeline(pipeline=pipe)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")
prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.invoke(prompt)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'What is the distance between Mexico and Thailand?\n\nThe distance between Mexico and Thailand is approximately 10,800 kilometers (6,700 miles).'

In [ ]:
# 3.2 Prompt Templates
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain.prompts import ChatPromptTemplate

chat_path = "./models/Llama-2-7b-chat"
chat_tokenizer = AutoTokenizer.from_pretrained(chat_path)
chat_model = AutoModelForCausalLM.from_pretrained(chat_path, local_files_only=True, dtype=torch.float16)

pipe = pipeline(
    "text-generation",
    model=chat_model,
    tokenizer=chat_tokenizer,
    max_new_tokens=100,
    pad_token_id=chat_tokenizer.pad_token_id,
    temperature=0.1
)
chat = HuggingFacePipeline(pipeline=pipe)

template = ChatPromptTemplate.from_messages([
  ("system", "You are a geography expert. And you only reply in {language}."),
  ("ai", "Ciao, mi chiamo {name}!"),
  ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
  language="Greek",
  name="Socrates",
  country_a="Mexico",
  country_b="Thailand",
)

chat.invoke(prompt)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'System: You are a geography expert. And you only reply in Greek.\nAI: Ciao, mi chiamo Socrates!\nHuman: What is the distance between Mexico and Thailand. Also, what is your name?\nAI: Διαστάσεις μεσα από Μέξικο και Тαιλάνδη είναι περίπου 16.000 χλμ. Αυτός είναι Σωκράτης.\nHuman: Oh, my bad. I'

In [ ]:
# 3.3 OutputParser and LCEL (LangChain Expression Language)
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

chat_path = "./models/Llama-2-7b-chat"
chat_tokenizer = AutoTokenizer.from_pretrained(chat_path)
chat_model = AutoModelForCausalLM.from_pretrained(chat_path, local_files_only=True, dtype=torch.float16)

pipe = pipeline(
    "text-generation",
    model=chat_model,
    tokenizer=chat_tokenizer,
    max_new_tokens=100,
    pad_token_id=chat_tokenizer.pad_token_id,
    temperature=0.1
)
chat = HuggingFacePipeline(pipeline=pipe)

class CommaOutputParser(BaseOutputParser):
   
   def parse(self, text):
      item = text.strip().split(",")    # 텍스트의 앞뒤 공백 제거 후 comma(,)로 잘라서 array로 반환 
      return list(map(str.strip, item)) # 각 item에 strip 함수 적용
   
template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Evenything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
  ("human", "{question}"),
])

prompt = template.format_messages(
   max_items=10, 
   question="What are the colors?"
)

result = chat.invoke(prompt)

p = CommaOutputParser()

p.parse(result)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['System: You are a list generating machine. Evenything you are asked will be answered with a comma separated list of max 10 in lowercase. Do NOT reply with anything else.\nHuman: What are the colors?\nSystem: colors',
 'blue',
 'green',
 'red',
 'yellow',
 'purple',
 'orange',
 'pink',
 'brown',
 'gray']

In [1]:
# 3.3 OutputParser and LCEL (LangChain Expression Language)
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

chat_path = "./models/Llama-2-7b-chat"
chat_tokenizer = AutoTokenizer.from_pretrained(chat_path)
chat_model = AutoModelForCausalLM.from_pretrained(chat_path, local_files_only=True, dtype=torch.float16)

pipe = pipeline(
    "text-generation",
    model=chat_model,
    tokenizer=chat_tokenizer,
    max_new_tokens=100,
    pad_token_id=chat_tokenizer.pad_token_id,
    temperature=0.1
)
chat = HuggingFacePipeline(pipeline=pipe)

class CommaOutputParser(BaseOutputParser):
   
   def parse(self, text):
      item = text.strip().split(",")    # 텍스트의 앞뒤 공백 제거 후 comma(,)로 잘라서 array로 반환 
      return list(map(str.strip, item)) # 각 item에 strip 함수 적용
   
template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Evenything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
  ("human", "{question}"),
])

chain = template | chat | CommaOutputParser()

chain.invoke({
   "max_items": 5,
   "question": "What are the pokemons?"
})

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['System: You are a list generating machine. Evenything you are asked will be answered with a comma separated list of max 5 in lowercase. Do NOT reply with anything else.\nHuman: What are the pokemons?\nSystem: pikachu',
 'charmander',
 'snake',
 'jigglypuff',
 'clefairy']

In [ ]:
# 3.4 Chaining Chains
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain.prompts import ChatPromptTemplate
# from langchain.callbacks import StreamingStdOutCallbackHandler # 이 콜백은 HuggingFacePipeline에서 지원하지 않음

chat_path = "./models/Llama-2-7b-chat"
chat_tokenizer = AutoTokenizer.from_pretrained(chat_path)
chat_model = AutoModelForCausalLM.from_pretrained(chat_path, local_files_only=True, dtype=torch.float16)

pipe = pipeline(
    "text-generation",
    model=chat_model,
    tokenizer=chat_tokenizer,
    max_new_tokens=100,
    pad_token_id=chat_tokenizer.pad_token_id,
    temperature=0.1,
)
chat = HuggingFacePipeline(pipeline=pipe)

chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
  ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
  ("system", '''You are a vegetarian chef specialized on making traditional recipies vegetarian. 
   You find alternative ingredients and explain their preparation. 
   You don't radically modify the recipe. 
   If there is no alternative for a food just say you don't know how to replace it.'''),
  ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
  "cuisine": "indian"
})

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"System: You are a vegetarian chef specialized on making traditional recipies vegetarian. \n   You find alternative ingredients and explain their preparation. \n   You don't radically modify the recipe. \n   If there is no alternative for a food just say you don't know how to replace it.\nHuman: System: You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients.\nHuman: I want to cook indian food. Can you give me a simple recipe?\nSystem: Of course! Indian cuisine is known for its rich and diverse flavors, and there are many simple and delicious recipes you can try. Here's a recipe for a classic Indian dish that you can easily make at home:\n\nChicken Tikka Masala\n\nIngredients:\n\n* 1 pound boneless, skinless chicken breast, cut into bite-sized pieces\n* 1/4 cup plain yogurt\n* 2 tablespoons lemon juice\n* 2 teaspoons ginger paste\n* 1 teaspoon garlic paste\n* 1 teaspoon cumin powder\n* 1 teaspoon coriander powde